In [ ]:
import numpy as np

import os
import os.path as op

import matplotlib.pyplot as plt

import nibabel as nib

import importlib

import interpolation as interp

In [ ]:
from nilearn import image

path_to_dir = "/Users/acionca/data/hcph-template/multivar-v00/derivatives/allInRef"
anat_files = interp.get_anat_filenames(path_to_dir, template_prefix="ayaya", pattern="Template.nii.gz")

exclude = ["ses-025", "ses-pilot019", "ses-pilot021"]
for excl in exclude:
    anat_files = [file for file in anat_files if excl not in file]

ses_id = ["\n".join([f"{i}", op.basename(file).split("_")[3].split("-")[-1]]) for i, file in enumerate(anat_files)]

concat = image.concat_imgs(anat_files, dtype=np.int16)

concat_mean = image.mean_img(concat)
concat_std = image.math_img("np.std(img, axis=-1)", img=concat)

concat_std.to_filename(f"/Users/acionca/data/hcph-template/std_img_N{len(anat_files)}.nii.gz")
concat_mean.to_filename(f"/Users/acionca/data/hcph-template/mean_img_N{len(anat_files)}.nii.gz")

concat_array = concat.get_fdata()
del concat

In [ ]:
slice_id = 189
square_x = [165, 175]
square_y = [165, 175]

draw_square_x = [square_x[0], square_x[1], square_x[1], square_x[0], square_x[0]]
draw_square_y = [square_y[0], square_y[0], square_y[1], square_y[1], square_y[0]]

fig, ax = plt.subplots(ncols=2, figsize=(15, 8))

ax[0].imshow(concat_array.mean(axis=-1)[..., slice_id], cmap="binary_r", vmin=0)
ax[1].imshow(concat_array.std(axis=-1)[..., slice_id], cmap="binary_r", vmin=0, vmax=100)

for one_ax in ax:
    one_ax.plot(draw_square_x, draw_square_y)

In [ ]:
square_array = concat_array[:, :, slice_id][:, square_x[0]:square_x[1]][square_y[0]:square_y[1]]
print(square_array.shape)


fig, ax = plt.subplots(ncols=2, figsize=(15, 8))

ax[0].imshow(square_array.mean(axis=-1), cmap="binary_r", vmin=0)
ax[1].imshow(square_array.std(axis=-1), cmap="binary_r", vmin=0)

ax[0].set_title("Zoom on blue square - mean intensity")
ax[1].set_title("Zoom on blue square - STD intensity")

fig, ax = plt.subplots(figsize=(15, 8))

val_per_ses = square_array.reshape((-1, square_array.shape[-1])).T
x_scatter = np.vstack([np.arange(val_per_ses.shape[0])]*val_per_ses.shape[1]).T

for i, val in enumerate(val_per_ses):
    val_to_scatter = val[val > 0]
    ax.scatter(np.full_like(val_to_scatter, i+1), val_to_scatter, color="tab:blue", s=100, alpha=.6, edgecolor="none")
    ax.plot([i+.8, i+1.2], [val_to_scatter.mean()]*2, c="k")

ax.plot([-.2, len(val_per_ses)+.2], [val_per_ses.mean()]*2, c="k", alpha=.3)

chut = ax.set_xticks(1+np.arange(len(val_per_ses)), labels=ses_id)
ax.set_xlabel("Session (with id on top)")
ax.set_ylabel("Voxel intensity")
#chut = ax.set_xticklabels(ses_id)
#ax.scatter(x_scatter, val_per_ses)

In [ ]:
importlib.reload(interp)

np.random.seed(220367)
#distances = np.random.normal(np.ones((2, 5))-.6, .1)
distances = np.random.normal(np.ones((100, 1))-.6, .1)
        
fig, axes = plt.subplots(figsize=(10, 6))
for offset, mark in zip([0, .1], ["s", "o"]):
    for d_kernel in [1, 2, 3]:
        norm_dist = interp.normalize_distances(distances, dist_kernel_order=d_kernel, offset=offset)

        dif = np.abs(norm_dist.max() - norm_dist.min())
        axes.scatter(distances.flatten(), norm_dist.flatten(), s=100, alpha=.3, ec="none", marker=mark, label=f"BSpline{d_kernel}, offset={offset:1.1f}, diff={dif:1.3f}")

axes.legend()
axes.set_xlabel("Measured distance")
axes.set_ylabel("Normalized distance weight")